#### LEVEL 3: Single Burst and Unit-level Relationships

(1) Single Burst Raster and Unit Order
    - Which units fired during a single burst?
    - When did they peak?
    - What is their temporal order?
    - Can I visualize a clear activation sequence?
        - IFR matrix for a burst and sort units by peak firing rate within that window.

(2)  Relative Timing and Firing Order
    Goals:
        Analyze when each unit fires relative to the population burst peak across bursts
        Add temporal precision to burst structure metrics for rank order correlations and functional role identification.
            - How early or late does a unit fire?
            - Do some units always fire early? Are they consistent?
            - Can we detect sequential recruitment or backbone structures?
        Needs:
            A list of time differences between:
                Each unit's peak activity
                Population peak in the same burst
        Tasks:
            - Build histograms of activation order
            - Identify early-firing units
            - Compare temporal dispersion of units across bursts
        Ideas for expansion:
            - Group units by quartiles of firing latency
            - Track variability in firing latency across bursts
            - Identify scaffold units by consistently early peaks
            - Identify functional units by consistent temporal correlations across bursts


In [1]:
# --- Set up imports and paths ---
import sys
from pathlib import Path
import pandas as pd

# Set the project root so Python can find analysis_libs and others
project_root = Path("~/bioinformatics").expanduser().resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Define the data directory
data_path = project_root / "data/extracted/maxtwo_newconfig1"

In [2]:
# -----------------------------
# 1. Load one dataset
# -----------------------------
data_path = Path("~/bioinformatics/data/extracted/maxtwo_newconfig1").expanduser()
from burst_analysis.loading import SpikeDataLoader

loader = SpikeDataLoader(data_path)
datasets = loader.load()  # dictionary of dataset_key -> SpikeData object

print(f"Datasets loaded: {list(datasets.keys())}")

Datasets loaded: ['M06359s1_D53_175µM_T1_1hr', 'M06359s1_D53_175µM_T1_3hr', 'M06359s1_D55_175µM_T2_24hr', 'M06359s1_D56_175µM_T2_48hr', 'M06359s1_D57_175µM_T2_72hr', 'M06359s2_D53_Control_T1_1hr', 'M06359s2_D53_Control_T1_3hr', 'M06359s2_D55_Control_T2_24hr', 'M06359s2_D56_Control_T2_48hr', 'M06359s2_D57_Control_T2_72hr', 'M06359s3_D53_175µM_T1_1hr', 'M06359s3_D53_175µM_T1_3hr', 'M06359s3_D55_175µM_T2_24hr', 'M06359s3_D56_175µM_T2_48hr', 'M06359s3_D57_175µM_T2_72hr', 'M06359s4_D53_175µM_T1_1hr', 'M06359s4_D53_175µM_T1_3hr', 'M06359s4_D55_175µM_T2_24hr', 'M06359s4_D56_175µM_T2_48hr', 'M06359s4_D57_175µM_T2_72hr', 'M06359s5_D53_175µM_T1_1hr', 'M06359s5_D53_175µM_T1_3hr', 'M06359s5_D55_175µM_T2_24hr', 'M06359s5_D56_175µM_T2_48hr', 'M06359s5_D57_175µM_T2_72hr', 'M06359s6_D53_175µM_T1_1hr', 'M06359s6_D53_175µM_T1_3hr', 'M06359s6_D55_175µM_T2_24hr', 'M06359s6_D56_175µM_T2_48hr', 'M06359s6_D57_175µM_T2_72hr', 'MO6359s1_D53_175µM_BASELINE_0hr', 'MO6359s1_D58_175µM_T2_D4', 'MO6359s1_D59_175µM_T

In [3]:
from projects.parkinsons.coordinator import OrchestratorPDx2

orc = OrchestratorPDx2(data_path)
print("Orchestrator initialized.")
print("Available dataset keys:", orc.list_datasets())

Orchestrator initialized.
Available dataset keys: ['M06359s1_D53_175µM_T1_1hr', 'M06359s1_D53_175µM_T1_3hr', 'M06359s1_D55_175µM_T2_24hr', 'M06359s1_D56_175µM_T2_48hr', 'M06359s1_D57_175µM_T2_72hr', 'M06359s2_D53_Control_T1_1hr', 'M06359s2_D53_Control_T1_3hr', 'M06359s2_D55_Control_T2_24hr', 'M06359s2_D56_Control_T2_48hr', 'M06359s2_D57_Control_T2_72hr', 'M06359s3_D53_175µM_T1_1hr', 'M06359s3_D53_175µM_T1_3hr', 'M06359s3_D55_175µM_T2_24hr', 'M06359s3_D56_175µM_T2_48hr', 'M06359s3_D57_175µM_T2_72hr', 'M06359s4_D53_175µM_T1_1hr', 'M06359s4_D53_175µM_T1_3hr', 'M06359s4_D55_175µM_T2_24hr', 'M06359s4_D56_175µM_T2_48hr', 'M06359s4_D57_175µM_T2_72hr', 'M06359s5_D53_175µM_T1_1hr', 'M06359s5_D53_175µM_T1_3hr', 'M06359s5_D55_175µM_T2_24hr', 'M06359s5_D56_175µM_T2_48hr', 'M06359s5_D57_175µM_T2_72hr', 'M06359s6_D53_175µM_T1_1hr', 'M06359s6_D53_175µM_T1_3hr', 'M06359s6_D55_175µM_T2_24hr', 'M06359s6_D56_175µM_T2_48hr', 'M06359s6_D57_175µM_T2_72hr', 'MO6359s1_D53_175µM_BASELINE_0hr', 'MO6359s1_D58_17

In [4]:
# --- 1) Define burst detection config (adjust per dataset if needed) ---
config = {
    "bin_size_ms": 5,
    "square_win_ms": 5,
    "gauss_win_ms": 17,
    "threshold_rms": 1.80,
    "min_dist_ms": 2000,
    "burst_edge_fraction": 0.25
}


# --- 2) Detect bursts for all datasets and cache results ---
burst_results = {}

for key in orc.list_datasets():
    detector = orc.get_burst_detector(key, config=config)
    result = detector.compute_population_rate_and_bursts()

    if result and result[-1]:
        burst_results[key] = {
            "detector": detector,
            "times": result[0],
            "smoothed_rate": result[1],
            "peaks": result[2],
            "peak_times": result[3],
            "bursts": result[4],
            "burst_windows": result[5]
        }
        print(f"{key}: {len(result[5])} bursts detected")
    else:
        print(f"{key}: No bursts detected (with current config)")


M06359s1_D53_175µM_T1_1hr: 309 bursts detected
M06359s1_D53_175µM_T1_3hr: 293 bursts detected
M06359s1_D55_175µM_T2_24hr: 357 bursts detected
M06359s1_D56_175µM_T2_48hr: 310 bursts detected
M06359s1_D57_175µM_T2_72hr: 350 bursts detected
M06359s2_D53_Control_T1_1hr: 157 bursts detected
M06359s2_D53_Control_T1_3hr: 160 bursts detected
M06359s2_D55_Control_T2_24hr: 199 bursts detected
M06359s2_D56_Control_T2_48hr: 202 bursts detected
M06359s2_D57_Control_T2_72hr: 172 bursts detected
M06359s3_D53_175µM_T1_1hr: 295 bursts detected
M06359s3_D53_175µM_T1_3hr: 261 bursts detected
M06359s3_D55_175µM_T2_24hr: 321 bursts detected
M06359s3_D56_175µM_T2_48hr: 308 bursts detected
M06359s3_D57_175µM_T2_72hr: 329 bursts detected
M06359s4_D53_175µM_T1_1hr: 309 bursts detected
M06359s4_D53_175µM_T1_3hr: 287 bursts detected
M06359s4_D55_175µM_T2_24hr: 340 bursts detected
M06359s4_D56_175µM_T2_48hr: 303 bursts detected
M06359s4_D57_175µM_T2_72hr: 354 bursts detected
M06359s5_D53_175µM_T1_1hr: 173 bursts 

In [ ]:
# load backbone and non-rigid groups from metrics.cav 

import pandas as pd
import json
from pathlib import Path

csv_path = Path("~/bioinformatics/projects/parkinsons/metrics.csv").expanduser()

def safe_json_load(x):

    """
    loads JSON strings and 
        returns original value if
            - it's not a string
            - it's empty
            - it's invalid JSON
    """

    if isinstance(x, str):
        x = x.strip()
        if x.startswith("[") and x.endswith("]"):
            try:
                return json.loads(x)
            except json.JSONDecodeError:
                return x  # return raw string if parsing fails
    return x

if csv_path.exists():
    df = pd.read_csv(csv_path)

    # try to load any JSON-like column
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].apply(safe_json_load)

    burst_extraction_records = df.to_dict(orient="records")
    print(f"Loaded {len(burst_extraction_records)} records from {csv_path}")
else:
    burst_extraction_records = []
    print("No saved metrics.csv found — starting fresh")

In [ ]:
# plot FR distributions for 3 putative backbone / non-rigid units in a specific burst (remember to make stacked
# unit rasters for comparison and also 2 or 3 plots of the same units in different bursts)

dataset_key = "M06359s1_D53_175µM_T1_1hr"
windows = burst_results[dataset_key]["burst_windows"]

fig = orc.plot_figure3_panel_a(
    dataset_key=dataset_key,
    unit_indices=[0, 1, 2],
    burst_idx=0,
    burst_windows=burst_results[dataset_key]["burst_windows"],
    bin_size=0.035,  # 50 ms 
    save=False
)
display(fig)

In [ ]:
# 1) build a quick df to inspect burst durations and identify nice plot candidates
'''
burst window is intentionally short and centered tightly around the population burst start and end, 
focusing on the single-burst firing rate dynamics of a few units. Based on the papers methods and 
the figure itself:
Window duration: ~200-250 ms total
Each burst in the figure starts with a sharp rise and decays to baseline within ~150-200 ms.
They likely selected bursts with clean onset/offset and cut the window just slightly longer than 
the population-defined burst start and end.
Bin size: 2-5 ms
This keeps the IFR curves smooth without being overly spiky.
The original figure likely used 5 ms square bins with Gaussian smoothing 
(Methods specify a 5 ms square window for burst peak timing).
'''

# Build a quick DataFrame of burst durations for a dataset
dataset_key = "M06359s1_D53_175µM_T1_1hr"
windows = burst_results[dataset_key]["burst_windows"]

burst_df = pd.DataFrame(windows, columns=["start", "end"])
burst_df["duration_ms"] = (burst_df["end"] - burst_df["start"]) * 1000

# View bursts sorted by duration
burst_df_sorted = burst_df.sort_values("duration_ms")
display(burst_df_sorted.head(20))

In [ ]:
# 2) filter for bursts in target range (~ 150-250 ms)

short_bursts = burst_df[(burst_df["duration_ms"] >= 150) &
                        (burst_df["duration_ms"] <= 250)]
print(f"Found {len(short_bursts)} candidate bursts")

#3) choose an ideal burst window for plotting in the next cell

# or just scroll through the metrics csv loaded at the top of the notebook

In [ ]:
# loop over candidate bursts and save 
output_dir = Path("~/bioinformatics/projects/parkinsons/results/fig3a_IFR_overlays").expanduser()
output_dir.mkdir(parents=True, exist_ok=True)

candidate_indices = short_bursts.index.tolist()

for idx in candidate_indices:   # candidate_indices or range(len(windows))
    start, end = windows[idx]
    save_path = output_dir / f"burst_{idx:03d}.png"
    fig = orc.plot_figure3_panel_a(
        dataset_key=dataset_key,
        unit_indices=[0, 1, 2],
        burst_idx=idx,
        burst_windows=windows,
        bin_size=0.005,
        save_path=save_path
    )

In [ ]:
# Run the unit burst peak centered FR plot on all of the above candidate bursts and save
# Output directory
output_dir = Path("~/bioinformatics/projects/parkinsons/results/fig3a_peak_centered_avgFRs").expanduser()
output_dir.mkdir(parents=True, exist_ok=True)

candidate_indices = short_bursts.index.tolist()

for idx in candidate_indices:
    start, end = windows[idx]
    save_path = output_dir / f"burst_{idx:03d}.png"
    fig = orc.plot_figure3_panel_a(
        dataset_key=dataset_key,
        unit_indices=[0, 1, 2],
        burst_idx=idx,
        burst_windows=windows,
        bin_size=0.005,
        save=True,
        save_path=save_path
    )
    print(f"Saved: {save_path}")

In [ ]:
dataset_key = "M06359s1_D53_175µM_T1_1hr"
unit_indices = [2, 10, 20]

# Retrieve cached detection results
windows = burst_results[dataset_key]["burst_windows"]
peaks = burst_results[dataset_key]["peak_times"]

# Output directory
output_dir = Path("~/bioinformatics/projects/parkinsons/results/fig3b_peak_centered_avgFRs").expanduser()
output_dir.mkdir(parents=True, exist_ok=True)

# Generate, save, and print figure
fig = orc.plot_figure3_panel_b(
    dataset_key=dataset_key,
    unit_indices=unit_indices,
    burst_windows=windows,
    burst_peaks=peaks,
    bin_size=0.005,
    save_path=output_dir / f"{dataset_key}_fig3b_overlay_units{','.join(map(str,unit_indices))}.png",
    config=None
)

if fig:
    plt.show()
else:
    print("No figure returned.")


In [5]:
# testing backbone detection method


dataset_key = "MO6359s2_D53_Control_BASELINE_0hr"

# Run detection using cached burst results or df
bb_units, nr_units = orc.run_backbone_detection(
    dataset_key=dataset_key,
    burst_results=burst_results,        # or metrics_df=my_loaded_df
    min_spikes_per_burst=2,
    min_fraction_bursts=0.8,            # 80% of bursts threshold
    min_total_spikes=30
)

print("\n=== Backbone Detection Test ===")
print(f"Dataset: {dataset_key}")
print(f"Total units: {len(orc.spike_data[dataset_key].train)}")
print(f"Backbone units ({len(bb_units)}): {bb_units}")
print(f"Non-rigid units ({len(nr_units)}): {nr_units}")


[Backbone Detection] Dataset: MO6359s2_D53_Control_BASELINE_0hr
Detected 143 backbone units: [1, 2, 3, 4, 6, 9, 11, 14, 15, 17, 19, 20, 21, 22, 24, 25, 26, 27, 30, 31, 32, 33, 34, 36, 39, 40, 43, 44, 45, 46, 47, 48, 54, 55, 57, 59, 60, 62, 65, 66, 67, 68, 70, 71, 74, 76, 77, 78, 79, 80, 82, 83, 86, 87, 88, 91, 93, 96, 100, 101, 102, 104, 106, 109, 110, 111, 115, 118, 119, 122, 124, 126, 127, 128, 129, 130, 132, 134, 135, 139, 141, 142, 143, 144, 145, 148, 151, 152, 154, 160, 164, 166, 168, 169, 172, 175, 176, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 205, 206, 207, 208, 209, 210, 211, 213, 215, 230, 236, 241, 242, 246, 248, 252, 263, 282, 283, 292, 294, 296, 299]
Detected 166 non-rigid units: [0, 5, 7, 8, 10, 12, 13, 16, 18, 23, 28, 29, 35, 37, 38, 41, 42, 49, 50, 51, 52, 53, 56, 58, 61, 63, 64, 69, 72, 73, 75, 81, 84, 85, 89, 90, 92, 94, 95, 97, 98, 99, 103, 105, 107, 108, 112, 113, 114, 116, 117, 120, 121, 123,

In [6]:
# debug backbone detection stats
import pandas as pd
from burst_analysis.computation import BurstAnalysisMicro

dataset_key = "MO6359s2_D53_Control_BASELINE_0hr"

# Retrieve cached burst windows
burst_windows = burst_results[dataset_key]["burst_windows"]

# Initialize analysis object
sd = orc.spike_data[dataset_key]
analysis = BurstAnalysisMicro(sd.train, fs=sd.metadata.get("fs", 10000))

# Collect per-unit stats
rows = []
for unit_idx, spike_train in enumerate(sd.train):
    total_spikes = 0
    active_bursts = 0

    for (start, end) in burst_windows:
        spikes_in_burst = spike_train[(spike_train >= start) & (spike_train <= end)]
        n_spikes = len(spikes_in_burst)
        total_spikes += n_spikes
        if n_spikes >= 2:
            active_bursts += 1

    fraction_active = active_bursts / len(burst_windows)
    classification = "Backbone" if (total_spikes >= 30 and fraction_active >= 0.8) else "Non-Rigid"

    rows.append({
        "Unit": unit_idx,
        "Total Spikes": total_spikes,
        "Active Bursts": active_bursts,
        "Total Bursts": len(burst_windows),
        "Fraction Active": round(fraction_active, 3),
        "Classification": classification
    })

# Convert to DataFrame for display
df_backbone_stats = pd.DataFrame(rows)

# Display table sorted by classification first, then total spikes
df_backbone_stats = df_backbone_stats.sort_values(
    by=["Classification", "Total Spikes"], ascending=[True, False]
).reset_index(drop=True)

# Show table
display(df_backbone_stats)


,Unit,Total Spikes,Active Bursts,Total Bursts,Fraction Active,Classification
0,24,7336,305,310,0.984,Backbone
1,14,5898,304,310,0.981,Backbone
2,194,5866,309,310,0.997,Backbone
3,144,5675,307,310,0.990,Backbone
4,25,5658,307,310,0.990,Backbone
...,...,...,...,...,...,...
304,228,54,6,310,0.019,Non-Rigid
305,265,49,8,310,0.026,Non-Rigid
306,266,48,11,310,0.035,Non-Rigid
307,7,33,10,310,0.032,Non-Rigid


In [ ]:
# quick test cell for panel c

import time
from pathlib import Path

dataset_key = "MO6359s2_D53_Control_BASELINE_0hr"
save_path = "results/test_fig3_panel_c_patched.png"

start = time.time()
fig_c = orc.plot_figure3_panel_c(
    dataset_key=dataset_key,
    burst_results=burst_results,
    min_spikes_per_burst=2,
    min_fraction_bursts=0.8,
    min_total_spikes=30,
    max_lag=0.35,
    save_path=save_path
)
elapsed = time.time() - start

print("\n=== Panel C Patched Test ===")
print(f"Runtime: {elapsed:.2f} sec")
print(f"Figure saved to: {Path(save_path).resolve()}")
if fig_c:
    print("Panel C figure generated successfully (grayscale style).")
    fig_c.show()
else:
    print("Panel C figure failed to generate.")


In [ ]:
# plot panel c

import time
from pathlib import Path

# === SETTINGS ===
dataset_key = "MO6359s2_D53_Control_BASELINE_0hr"
output_path = "results/Figure3_PanelC_final.png"

# Ensure results folder exists
Path("results").mkdir(parents=True, exist_ok=True)

# === RUN PANEL C ===
start = time.time()
fig_panel_c = orc.plot_figure3_panel_c(
    dataset_key=dataset_key,
    burst_results=burst_results,
    min_spikes_per_burst=2,
    min_fraction_bursts=0.8,
    min_total_spikes=30,
    max_lag=0.35,
    save_path=output_path
)
elapsed = time.time() - start

# === REPORT RESULTS ===
print("\n=== FINAL FIGURE 3 PANEL C ===")
print(f"Dataset: {dataset_key}")
print(f"Runtime: {elapsed:.2f} sec")
print(f"Figure saved to: {Path(output_path).resolve()}")

if fig_panel_c:
    print("Panel C successfully generated in target style.")
    fig_panel_c.show()
else:
    print("Panel C generation failed.")


In [ ]:
# plot panel d

import time
from pathlib import Path

dataset_key = "MO6359s2_D53_Control_BASELINE_0hr"
output_path = "results/Figure3_PanelD_final.png"

Path("results").mkdir(parents=True, exist_ok=True)

start = time.time()
fig_panel_d = orc.plot_figure3_panel_d(
    dataset_key=dataset_key,
    burst_results=burst_results,
    min_spikes_per_burst=2,
    min_fraction_bursts=0.8,
    min_total_spikes=30,
    max_lag=0.35,
    save_path=output_path
)
elapsed = time.time() - start

print("\n=== FINAL FIGURE 3 PANEL D ===")
print(f"Dataset: {dataset_key}")
print(f"Runtime: {elapsed:.2f} sec")
print(f"Figure saved to: {Path(output_path).resolve()}")

if fig_panel_d:
    print("Panel D successfully generated in target style.")
    fig_panel_d.show()
else:
    print("Panel D generation failed.")


In [ ]:
# plot panel e

dataset_key = "MO6359s2_D53_Control_BASELINE_0hr"

fig_e = orc.plot_figure3_panel_e(
    dataset_key=dataset_key,
    burst_results=burst_results,
    min_spikes_per_burst=2,
    min_fraction_bursts=0.8,
    min_total_spikes=30,
    max_lag=0.35,
    save_path="results/fig3_panel_e_violin.png"
)

if fig_e:
    print("Panel E figure generated successfully.")
    fig_e.show()
else:
    print("Panel E failed to generate.")


In [ ]:
# plot panel f

dataset_key = "MO6359s2_D53_Control_BASELINE_0hr"

fig_f = orc.plot_figure3_panel_f(
    dataset_key=dataset_key,
    burst_results=burst_results,
    min_spikes_per_burst=2,
    min_fraction_bursts=0.8,
    min_total_spikes=30,
    max_lag=0.35,
    save_path="results/fig3_panel_f_histogram.png"
)

if fig_f:
    print("Panel F figure generated successfully.")
    fig_f.show()
else:
    print("Panel F failed to generate.")
